# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Call path to csv 
city_data = "./Output_data/cities.csv"
# Read Cities Data File and store into Pandas Data Frames
city_df = pd.read_csv(city_data)
city_df.head(5)

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,chapais,90,CA,1569459446,87,49.78,-74.86,50.00,13.87
1,1,nelson bay,32,CA,1569459446,36,49.49,-117.29,57.19,3.42
2,2,hermanus,0,ZA,1569459446,68,-34.42,19.24,66.99,10.00
3,3,tsuyama,0,JP,1569459447,64,35.14,134.03,71.60,5.82
4,4,taywarah,0,AF,1569459447,22,33.35,64.42,53.82,6.38


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
#gmaps.configure(api_key=gkey)
# Store 'Lat' and 'Lng' into  locations 
locations = city_df[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
humidity = city_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Conditions to meet A max temperature lower than 80 degrees but higher than 70, Wind speed less than 10 mph, zero cloudiness

# Create variable with TRUE if age is greater than 50
temp = city_df['Max Temp'] < 75
wind = city_df['Wind Speed'] < 5
clouds = city_df['Cloudiness'] < .1
humidity = city_df['Humidity'] < 60

hotel_df = city_df[temp & wind & clouds & humidity]
hotel_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
123,123,guerrero negro,0,AR,1569459503,40,-24.19,-65.45,68.00,0.98
176,176,mar del plata,0,CN,1569459525,43,29.86,90.73,29.51,2.68
319,319,progreso,0,UY,1569459589,59,-34.68,-56.22,62.60,4.70
322,322,cherskiy,0,RU,1569459591,54,68.75,161.30,34.35,4.61
339,339,mafeteng,0,LS,1569459598,54,-29.82,27.24,32.49,2.77
357,357,qaqortoq,0,GL,1569459620,51,60.72,-46.03,42.80,3.36
370,370,akdepe,0,TM,1569459626,54,42.05,59.39,55.40,4.70
398,398,miraflores,0,PE,1569459642,25,-16.39,-71.52,60.80,2.24
403,403,mitu,0,IR,1569459643,46,36.20,46.09,52.98,3.65
423,423,narsaq,0,GL,1569459489,51,60.91,-46.05,42.80,3.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name']= ""
hotel_df

/Users/williewiedenmann/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
123,123,guerrero negro,0,AR,1569459503,40,-24.19,-65.45,68.00,0.98,
176,176,mar del plata,0,CN,1569459525,43,29.86,90.73,29.51,2.68,
319,319,progreso,0,UY,1569459589,59,-34.68,-56.22,62.60,4.70,
322,322,cherskiy,0,RU,1569459591,54,68.75,161.30,34.35,4.61,
339,339,mafeteng,0,LS,1569459598,54,-29.82,27.24,32.49,2.77,
357,357,qaqortoq,0,GL,1569459620,51,60.72,-46.03,42.80,3.36,
370,370,akdepe,0,TM,1569459626,54,42.05,59.39,55.40,4.70,
398,398,miraflores,0,PE,1569459642,25,-16.39,-71.52,60.80,2.24,
403,403,mitu,0,IR,1569459643,46,36.20,46.09,52.98,3.65,
423,423,narsaq,0,GL,1569459489,51,60.91,-46.05,42.80,3.36,


In [7]:
# geocoordinates
target_coordinates = "-24.19 ,-65.45"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
response = requests.get(base_url, params=params)
places_data = response.json()
hotel_name = places_data["results"][0]["name"]

hotel_df.loc[[123], 'Hotel Name'] = hotel_name
hotel_df



/Users/williewiedenmann/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
123,123,guerrero negro,0,AR,1569459503,40,-24.19,-65.45,68.00,0.98,Termas de Reyes
176,176,mar del plata,0,CN,1569459525,43,29.86,90.73,29.51,2.68,
319,319,progreso,0,UY,1569459589,59,-34.68,-56.22,62.60,4.70,
322,322,cherskiy,0,RU,1569459591,54,68.75,161.30,34.35,4.61,
339,339,mafeteng,0,LS,1569459598,54,-29.82,27.24,32.49,2.77,
357,357,qaqortoq,0,GL,1569459620,51,60.72,-46.03,42.80,3.36,
370,370,akdepe,0,TM,1569459626,54,42.05,59.39,55.40,4.70,
398,398,miraflores,0,PE,1569459642,25,-16.39,-71.52,60.80,2.24,
403,403,mitu,0,IR,1569459643,46,36.20,46.09,52.98,3.65,
423,423,narsaq,0,GL,1569459489,51,60.91,-46.05,42.80,3.36,


In [8]:
# geocoordinates
target_coordinates = "29.86, 90.73"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
#"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY"
# run a request using our params dictionary
response = requests.get(base_url, params=params)
places_data = response.json()
print(json.dumps(places_data, indent=4, sort_keys=True))
#hotel_name = places_data["results"][0]["name"]
#print(hotel_name)
#df.loc[[0,3],'Z'] = 3

#hotel_df.loc[[176], 'Hotel Name'] = hotel_name


{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}


In [9]:
# Due to the 0 results above, adding a NO HOTEL to the hotel name column 
hotel_df.loc[[176], 'Hotel Name'] = 'No Hotel'
hotel_df.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
123,123,guerrero negro,0,AR,1569459503,40,-24.19,-65.45,68.00,0.98,Termas de Reyes
176,176,mar del plata,0,CN,1569459525,43,29.86,90.73,29.51,2.68,No Hotel
319,319,progreso,0,UY,1569459589,59,-34.68,-56.22,62.60,4.70,
322,322,cherskiy,0,RU,1569459591,54,68.75,161.30,34.35,4.61,
339,339,mafeteng,0,LS,1569459598,54,-29.82,27.24,32.49,2.77,


In [10]:
# geocoordinates
target_coordinates = "-34.68, -56.22"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
#"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY"
# run a request using our params dictionary
response = requests.get(base_url, params=params)
places_data = response.json()
#print(json.dumps(places_data, indent=4, sort_keys=True))
hotel_name = places_data["results"][0]["name"]
#print(hotel_name)
#df.loc[[0,3],'Z'] = 3

hotel_df.loc[[319], 'Hotel Name'] = hotel_name
hotel_df.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
123,123,guerrero negro,0,AR,1569459503,40,-24.19,-65.45,68.00,0.98,Termas de Reyes
176,176,mar del plata,0,CN,1569459525,43,29.86,90.73,29.51,2.68,No Hotel
319,319,progreso,0,UY,1569459589,59,-34.68,-56.22,62.60,4.70,Villa La Alegria
322,322,cherskiy,0,RU,1569459591,54,68.75,161.30,34.35,4.61,
339,339,mafeteng,0,LS,1569459598,54,-29.82,27.24,32.49,2.77,


In [11]:
# geocoordinates
target_coordinates = "35.77, 102.37"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
#"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY"
# run a request using our params dictionary
response = requests.get(base_url, params=params)
places_data = response.json()
print(json.dumps(places_data, indent=4, sort_keys=True))
#hotel_name = places_data["results"][0]["name"]
#print(hotel_name)
#df.loc[[0,3],'Z'] = 3

#hotel_df.loc[[70], 'Hotel Name'] = hotel_name


{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}


In [12]:
#Due to 0 results, adding No Hotel name 
# Due to the 0 results above, adding a NO HOTEL to the hotel name column 
hotel_df.loc[[322], 'Hotel Name'] = 'No Hotel'
hotel_df.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
123,123,guerrero negro,0,AR,1569459503,40,-24.19,-65.45,68.00,0.98,Termas de Reyes
176,176,mar del plata,0,CN,1569459525,43,29.86,90.73,29.51,2.68,No Hotel
319,319,progreso,0,UY,1569459589,59,-34.68,-56.22,62.60,4.70,Villa La Alegria
322,322,cherskiy,0,RU,1569459591,54,68.75,161.30,34.35,4.61,No Hotel
339,339,mafeteng,0,LS,1569459598,54,-29.82,27.24,32.49,2.77,


In [13]:
# geocoordinates
target_coordinates = "-29.82, 27.24"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
#"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY"
# run a request using our params dictionary
response = requests.get(base_url, params=params)
places_data = response.json()
#print(json.dumps(places_data, indent=4, sort_keys=True))
hotel_name = places_data["results"][0]["name"]
#print(hotel_name)
#df.loc[[0,3],'Z'] = 3

hotel_df.loc[[339], 'Hotel Name'] = hotel_name
hotel_df.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
123,123,guerrero negro,0,AR,1569459503,40,-24.19,-65.45,68.00,0.98,Termas de Reyes
176,176,mar del plata,0,CN,1569459525,43,29.86,90.73,29.51,2.68,No Hotel
319,319,progreso,0,UY,1569459589,59,-34.68,-56.22,62.60,4.70,Villa La Alegria
322,322,cherskiy,0,RU,1569459591,54,68.75,161.30,34.35,4.61,No Hotel
339,339,mafeteng,0,LS,1569459598,54,-29.82,27.24,32.49,2.77,Golden Hotel


In [14]:
# geocoordinates
target_coordinates = "60.72, -46.03"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
#"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY"
# run a request using our params dictionary
response = requests.get(base_url, params=params)
places_data = response.json()
#print(json.dumps(places_data, indent=4, sort_keys=True))
hotel_name = places_data["results"][0]["name"]
#print(hotel_name)
#df.loc[[0,3],'Z'] = 3

hotel_df.loc[[357], 'Hotel Name'] = hotel_name
hotel_df.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
123,123,guerrero negro,0,AR,1569459503,40,-24.19,-65.45,68.00,0.98,Termas de Reyes
176,176,mar del plata,0,CN,1569459525,43,29.86,90.73,29.51,2.68,No Hotel
319,319,progreso,0,UY,1569459589,59,-34.68,-56.22,62.60,4.70,Villa La Alegria
322,322,cherskiy,0,RU,1569459591,54,68.75,161.30,34.35,4.61,No Hotel
339,339,mafeteng,0,LS,1569459598,54,-29.82,27.24,32.49,2.77,Golden Hotel


In [15]:
# geocoordinates
target_coordinates = "42.05, 59.39"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
#"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY"
# run a request using our params dictionary
response = requests.get(base_url, params=params)
places_data = response.json()
print(json.dumps(places_data, indent=4, sort_keys=True))
#hotel_name = places_data["results"][0]["name"]
#print(hotel_name)

#hotel_df.loc[[370], 'Hotel Name'] = hotel_name
#hotel_df.head()


{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}


In [16]:
#Due to 0 results, adding No Hotel name 
# Due to the 0 results above, adding a NO HOTEL to the hotel name column 
hotel_df.loc[[370], 'Hotel Name'] = 'No Hotel'
hotel_df.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
123,123,guerrero negro,0,AR,1569459503,40,-24.19,-65.45,68.00,0.98,Termas de Reyes
176,176,mar del plata,0,CN,1569459525,43,29.86,90.73,29.51,2.68,No Hotel
319,319,progreso,0,UY,1569459589,59,-34.68,-56.22,62.60,4.70,Villa La Alegria
322,322,cherskiy,0,RU,1569459591,54,68.75,161.30,34.35,4.61,No Hotel
339,339,mafeteng,0,LS,1569459598,54,-29.82,27.24,32.49,2.77,Golden Hotel


In [17]:
# geocoordinates
target_coordinates = "-16.39, -71.52"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
#"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY"
# run a request using our params dictionary
response = requests.get(base_url, params=params)
places_data = response.json()
#print(json.dumps(places_data, indent=4, sort_keys=True))
hotel_name = places_data["results"][0]["name"]
#print(hotel_name)

hotel_df.loc[[398], 'Hotel Name'] = hotel_name


In [18]:
# geocoordinates
target_coordinates = "36.20, 46.09"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
#"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY"
# run a request using our params dictionary
response = requests.get(base_url, params=params)
places_data = response.json()
print(json.dumps(places_data, indent=4, sort_keys=True))
#hotel_name = places_data["results"][0]["name"]
#print(hotel_name)

#hotel_df.loc[[403], 'Hotel Name'] = hotel_name


{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}


In [19]:
#Due to 0 results, adding No Hotel name 
# Due to the 0 results above, adding a NO HOTEL to the hotel name column 
hotel_df.loc[[403], 'Hotel Name'] = 'No Hotel'


In [20]:
# geocoordinates
target_coordinates = "60.91, -46.05"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
#"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY"
# run a request using our params dictionary
response = requests.get(base_url, params=params)
places_data = response.json()
#print(json.dumps(places_data, indent=4, sort_keys=True))
hotel_name = places_data["results"][0]["name"]
#print(hotel_name)

hotel_df.loc[[423], 'Hotel Name'] = hotel_name


In [21]:
# geocoordinates
target_coordinates = "50.11, 129.44"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
#"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY"
# run a request using our params dictionary
response = requests.get(base_url, params=params)
places_data = response.json()
#print(json.dumps(places_data, indent=4, sort_keys=True))
hotel_name = places_data["results"][0]["name"]
#print(hotel_name)

hotel_df.loc[[426], 'Hotel Name'] = hotel_name

hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
123,123,guerrero negro,0,AR,1569459503,40,-24.19,-65.45,68.00,0.98,Termas de Reyes
176,176,mar del plata,0,CN,1569459525,43,29.86,90.73,29.51,2.68,No Hotel
319,319,progreso,0,UY,1569459589,59,-34.68,-56.22,62.60,4.70,Villa La Alegria
322,322,cherskiy,0,RU,1569459591,54,68.75,161.30,34.35,4.61,No Hotel
339,339,mafeteng,0,LS,1569459598,54,-29.82,27.24,32.49,2.77,Golden Hotel
357,357,qaqortoq,0,GL,1569459620,51,60.72,-46.03,42.80,3.36,Hotel Qaqortoq
370,370,akdepe,0,TM,1569459626,54,42.05,59.39,55.40,4.70,No Hotel
398,398,miraflores,0,PE,1569459642,25,-16.39,-71.52,60.80,2.24,Casa de Avila Hotel
403,403,mitu,0,IR,1569459643,46,36.20,46.09,52.98,3.65,No Hotel
423,423,narsaq,0,GL,1569459489,51,60.91,-46.05,42.80,3.36,Hotel Narsaq


In [25]:
#Plot the Marker layer on the gmap 
locations = [(-24.19, -65.45), (29.86, 90.73), (-34.68, -56.22), (68.75, 161.30), (-29.82, 27.24), (60.72, -46.03), (42.05, 59.39), (-16.39, -71.52), (36.20, 46.09), (60.91, -46.05), (50.11, 129.44)]
list_of_infoboxes = ['Hotel: Termas de Reyes City: guerrero negro Country: AR ', 
                    'Hotel: No Hotel City: mar del plata Country: CN ',
                    'Hotel: Villa La Alegria City: progreso Country: UY ',
                    'Hotel: No Hotel City: cherskiy  Country: RU ',
                    'Hotel: Golden Hotel City: mafeteng Country: LS ',
                    'Hotel: Hotel Qaqortoq City: qaqortoq Country: GL ',
                    'Hotel: No Hotel City: akdepe Country: TM ',
                    'Hotel: Casa de Avila Hotel City: miraflores Country: PE ',
                    'Hotel: No Hotel City: mitu Country: LU ',
                    'Hotel: Lyuks City: narsaq Country: GL ',
                    'Hotel: Lyuks City: zavitinsk Country: RU ']
marker_layer = gmaps.marker_layer(locations, info_box_content= list_of_infoboxes)
#, info_box_content= list_of_infoboxes)
fig.add_layer(marker_layer)

# Display Map
fig

#fig = gmaps.figure()


#fig.add_layer(heat_layer)

#fig

Figure(layout=FigureLayout(height='420px'))

# ===Don't execute below this line, just practice loops on scrap code =====

In [ ]:
#was trying to work on this for loop but couldnt get it to work 
for index, row in hotel_df.iterrows():
    
# geocoordinates
    target_coordinates = hotel_df[["Lat", "Lng"]]
    target_radius = 5000
    target_type = "lodging"
#row_loc = 

# set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
#"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY"
# run a request using our params dictionary
    response = requests.get(base_url, params=params)
    places_data = response.json()
#print(json.dumps(places_data, indent=4, sort_keys=True))
    try:
        hotel_name = places_data["results"][0]["name"]
    except: 
        pass
#print(hotel_name)
#df.loc[[0,3],'Z'] = 3
    hotel_df['Hotel Name'] = hotel_name
    hotel_df.loc[[index], 'Hotel Name'] = hotel_name

hotel_df





In [ ]:
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    target_coordinates = hotel_df[["Lat", "Lng"]]
    target_radius = 5000
    target_type = "lodging"

    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotel_name = hotel_name.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "Missing field/result... skipping."
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
Data = {'Lat': ['-24.19'], 'Long' : ['-65.45']}
df = pd.DataFrame(Data)
#df = pd.to_numeric(df['Location'], errors='coerce')
df.astype(float)


In [ ]:
lat = df['Lat'][0]
long = df['Long'][0]
lat

In [ ]:
locations = [(-24.19, -65.45), (29.86, 90.73), (-34.68, -56.22), (68.75, 161.30), (-29.82, 27.24), (60.72, -46.03), (42.05, 59.39), (-16.39, -71.52), (36.20, 46.09), (60.91, -46.05), (50.11, 129.44)]
list_of_infoboxes = ['Hotel: Termas de Reyes City: guerrero negro Country: AR ', 
                    'Hotel: No Hotel City: mar del plata Country: CN ',
                    'Hotel: Villa La Alegria City: progreso Country: UY ',
                    'Hotel: No Hotel City: cherskiy  Country: RU ',
                    'Hotel: Golden Hotel City: mafeteng Country: LS ',
                    'Hotel: Hotel Qaqortoq City: qaqortoq Country: GL ',
                    'Hotel: No Hotel City: akdepe Country: TM ',
                    'Hotel: Casa de Avila Hotel City: miraflores Country: PE ',
                    'Hotel: No Hotel City: mitu Country: LU ',
                    'Hotel: Lyuks City: narsaq Country: GL ',
                    'Hotel: Lyuks City: zavitinsk Country: RU ']
marker_layer = gmaps.symbol_layer(locations, fill_color='red', info_box_content= list_of_infoboxes)
#, info_box_content= list_of_infoboxes)
fig.add_layer(marker_layer)

# Display Map
fig

#fig = gmaps.figure()


#fig.add_layer(heat_layer)

#fig